# 1002

https://github.com/ultralytics/yolov5.git

### hhw 데이터셋 가져오기

* 안전모 데이터셋: https://public.roboflow.com/object-detection/hard-hat-workers

https://public.roboflow.com/ds/HSzjb19j05?key=mYws4wS3W4

In [ ]:
!curl -L "https://public.roboflow.com/ds/MFc61zv5Nv?key=XBDCyv8XpS" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
import glob
import random
from IPython.display import Image, display

file = random.choice(glob.glob("/content/yolov5/hhw/test/labels/*.txt"))
with open(file, 'r') as file:
    content = file.read()
    print(content)

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob("/content/yolov5/runs/detect/exp5/*.jpg"))
display(Image(filename=image_name))

In [1]:
from glob import glob

train_img_list = glob('/content/yolov5/hhw/train/images/*.jpg')
test_img_list = glob('/content/yolov5/hhw/test/images/*.jpg')

print(f"Number of training images: {len(train_img_list)}")
print(f"Number of testing images: {len(test_img_list)}")

Number of training images: 0


In [ ]:
from sklearn.model_selection import train_test_split

test_img_list, val_img_list = train_test_split(test_img_list, test_size=0.5, random_state=777)
print(f"Number of validation images: {len(val_img_list)}")
print(f"Number of testing images: {len(test_img_list)}")

In [ ]:
import yaml

with open('/content/yolov5/hhw/train.txt', 'w') as file:
    file.write('\n'.join(train_img_list) + '\n')
    
with open('/content/yolov5/hhw/val.txt', 'w') as file:
    file.write('\n'.join(val_img_list) + '\n')
    
with open('/content/yolov5/hhw/test.txt', 'w') as file:
    file.write('\n'.join(test_img_list) + '\n')
    


In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        

In [ ]:
%%writetemplate /content/yolov5/hhw/data/data.yaml
train: ./hhw/train.txt
test: ./hhw/test.txt
val: ./hhw/val.txt

nc: 3
names: ['head', 'helmet', 'person']

In [2]:
with open('/content/yolov5/hhw/data.yaml', 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
    print(f"number of classes: {num_classes}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov5/hhw/data.yaml'

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml
# Ultralytics YOLOv5 🚀, AGPL-3.0 license

# Parameters
nc: {num_classes} # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]


In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 64 --epochs 30 --data ./hhw/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name hhw_result --cache

In [ ]:
!ls /content/yolov5/runs/train/hhw_result


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/train/hhw_result

In [ ]:
Image(filename='/content/yolov5/runs/train/hhw_result/train_batch0.jpg', width=900)

In [ ]:
Image(filename="/content/yolov5/runs/train/hhw_result/val_batch0_labels.jpg", width=900)

In [ ]:
!python val.py --weights /content/yolov5/runs/train/hhw_result/weights/best.pt --data ./hhw/data.yaml --img 416 --iou 0.65

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/hhw_result/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/hhw/test/images

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob("/content/yolov5/runs/detect/exp6/*.jpg"))
display(Image(filename=image_name))